<img src="https://techla.pro/wp-content/uploads/2022/01/r5.png" width="240" height="240" align="right"/>

<center><h1>Grupo R5</header1></center>
<center><h>Prueba de selección: Reducir perdidas por fraude en las reclamaciones de siniestro</header2></center>

Presentado por: Daniela Gómez Sanchez  <br>
Fecha: 15/08/2022

## Importación de librerias necesarias

In [310]:
import psycopg2
import pandas as pd
import dtale
import numpy as np
import seaborn as sns
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

#### Modelo
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

## Conección a base de datos postgres

In [103]:
con = psycopg2.connect(database="postgres", user="postgres", password="", host="localhost", port="5432")
print("Database opened successfully")

Database opened successfully


In [104]:
#Utilizar cursor
cursor=con.cursor()

In [105]:
#Creaciión de sentencia
sql="SELECT monthh, Make, AccidentArea, DayOfWeekClaimed, MonthClaimed, WeekOfMonthClaimed, Age, Fault, PolicyType, VehicleCategory, VehiclePrice, FraudFound_P, Deductible, Days_Policy_Accident, Days_Policy_Claim, PastNumberOfClaims, AgeOfVehicle, AgeOfPolicyHolder, PoliceReportFiled, AgentType, NumberOfSuppliments, NumberOfCars, Yearr, BasePolicy FROM fraudes"

In [106]:
#uso del metodo execute
cursor.execute(sql)

In [107]:
#mostrar resultados
registros=cursor.fetchall()
print(registros)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [102]:
#Cerrar conexiones
cursor.close()
con.close()

## Tratamiento de Base

### conversión de base a data frame

In [112]:
df = pd.read_sql_query(sql,con=con)

C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [113]:
df.head()

,monthh,make,accidentarea,dayofweekclaimed,monthclaimed,weekofmonthclaimed,age,fault,policytype,vehiclecategory,...,days_policy_claim,pastnumberofclaims,ageofvehicle,ageofpolicyholder,policereportfiled,agenttype,numberofsuppliments,numberofcars,yearr,basepolicy
0,Dec,Honda,Urban,Tuesday,Jan,1,21,Policy Holder,Sport - Liability,Sport,...,more than 30,none,3 years,26 to 30,No,External,none,3 to 4,1994,Liability
1,Jan,Honda,Urban,Monday,Jan,4,34,Policy Holder,Sport - Collision,Sport,...,more than 30,none,6 years,31 to 35,Yes,External,none,1 vehicle,1994,Collision
2,Oct,Honda,Urban,Thursday,Nov,2,47,Policy Holder,Sport - Collision,Sport,...,more than 30,1,7 years,41 to 50,No,External,none,1 vehicle,1994,Collision
3,Jun,Toyota,Rural,Friday,Jul,1,65,Third Party,Sedan - Liability,Sport,...,more than 30,1,more than 7,51 to 65,Yes,External,more than 5,1 vehicle,1994,Liability
4,Jan,Honda,Urban,Tuesday,Feb,2,27,Third Party,Sport - Collision,Sport,...,more than 30,none,5 years,31 to 35,No,External,none,1 vehicle,1994,Collision


### Análisis descriptivo

In [119]:
#Infomación del data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30840 entries, 0 to 30839
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   monthh                30840 non-null  object
 1   make                  30840 non-null  object
 2   accidentarea          30840 non-null  object
 3   dayofweekclaimed      30840 non-null  object
 4   monthclaimed          30840 non-null  object
 5   weekofmonthclaimed    30840 non-null  int64 
 6   age                   30840 non-null  int64 
 7   fault                 30840 non-null  object
 8   policytype            30840 non-null  object
 9   vehiclecategory       30840 non-null  object
 10  vehicleprice          30840 non-null  object
 11  fraudfound_p          30840 non-null  int64 
 12  deductible            30840 non-null  int64 
 13  days_policy_accident  30840 non-null  object
 14  days_policy_claim     30840 non-null  object
 15  pastnumberofclaims    30840 non-null

In [117]:
#variables que contienen N/A
df.isnull().any()

monthh                  False
make                    False
accidentarea            False
dayofweekclaimed        False
monthclaimed            False
weekofmonthclaimed      False
age                     False
fault                   False
policytype              False
vehiclecategory         False
vehicleprice            False
fraudfound_p            False
deductible              False
days_policy_accident    False
days_policy_claim       False
pastnumberofclaims      False
ageofvehicle            False
ageofpolicyholder       False
policereportfiled       False
agenttype               False
numberofsuppliments     False
numberofcars            False
yearr                   False
basepolicy              False
dtype: bool

In [124]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
weekofmonthclaimed,30840.0,2.693969,1.259095,1.0,2.0,3.0,4.0,5.0
age,30840.0,39.855707,13.492158,0.0,31.0,38.0,48.0,80.0
fraudfound_p,30840.0,0.059857,0.237226,0.0,0.0,0.0,0.0,1.0
deductible,30840.0,407.704280,43.950285,300.0,400.0,400.0,400.0,700.0
yearr,30840.0,1994.866472,0.803300,1994.0,1994.0,1995.0,1996.0,1996.0


C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\views.py:1929: FutureWarning:

The default value of regex will change from True to False in a future version.



Exception occurred while processing request: Can only use .str accessor with string values!
Traceback (most recent call last):
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\views.py", line 2708, in get_column_analysis
    return jsonify(**analysis.build())
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\column_analysis.py", line 446, in build
    hist = self.build_hist(s, code)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\column_analysis.py", line 499, in build_hist
    pd.value_counts(s.str.split(expand=True).stack())
  File "

2022-08-15 13:11:35,567 - ERROR    - Exception occurred while processing request: Can only use .str accessor with string values!
Traceback (most recent call last):
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\views.py", line 2708, in get_column_analysis
    return jsonify(**analysis.build())
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\column_analysis.py", line 137, in build
    return_data, code = self.analysis.build(self)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\column_analysis.py", line 446, in build
    hist = self.build_hist(s, code)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\dtale\column_analysis.py", line 499, in build_hist
    pd.value_counts(s.str

In [160]:
desc=ProfileReport(df, explorative=True)
desc.to_file('Reporte_descriptivas.html')

Export report to file: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.12it/s]


In [147]:
fraudes=pd.crosstab (index = df ['fraudfound_p'], columns = ' recuento ')
fraudes=fraudes/fraudes.sum()
fraudes

col_0,recuento
fraudfound_p,
0,0.940143
1,0.059857


In [246]:
a=pd.DataFrame(df['fraudfound_p'].value_counts())
a.reset_index(inplace=True)

a.plot(kind='pie',y='fraudfound_p',labels=a['index'],autopct='%1.2f')
plt.title('Proporción en fraudes', fontsize = 20)
plt.show()

In [251]:
SeguroFraude=pd.crosstab(df.deductible,df.fraudfound_p)
SeguroFraude.plot(kind = 'bar')
plt.show()

In [254]:
SeguroVC=pd.crosstab(df.deductible, df.fault)
SeguroVC.plot(kind = 'barh')
plt.show()

In [252]:
SeguroVC=pd.crosstab(df.deductible,df.vehiclecategory)
colors = plt.cm.summer(np.linspace(0, 1, 5))
SeguroVC.div(SeguroVC.sum(1).astype(float), axis = 0).plot(kind = 'bar',
                                                           stacked = False,
                                                           figsize = (15, 7),
                                                           color = colors)

plt.title('Costo de seguro por categoría del vehículo', fontsize = 20)
plt.legend()
plt.show()

In [218]:
sns.boxplot(x=df.policytype,y=df.age,data=df)
plt.show()

Traceback (most recent call last):
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\cbook\__init__.py", line 287, in process
    func(*args, **kwargs)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\backend_bases.py", line 3279, in <lambda>
    "close_event", lambda e: manager.destroy())
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\site-packages\matplotlib\backends\_backend_tk.py", line 495, in destroy
    self._window_dpi.trace_remove('write', self._window_dpi_cbname)
  File "C:\Users\Holmes\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 455, in trace_remove
    self._tk.deletecommand(cbname)
_tkinter.TclError: can't delete Tcl command


In [231]:
pd.pivot_table (df,index='policytype', columns='fraudfound_p',aggfunc={'age': np.mean})


age           
fraudfound_p                  0          1
policytype                                
Sedan - All Perils    42.005441  38.420925
Sedan - Collision     38.784423  38.226562
Sedan - Liability     39.545950  38.666667
Sport - All Perils    33.545455        NaN
Sport - Collision     36.080000  34.166667
Sport - Liability     21.000000        NaN
Utility - All Perils  45.468227  40.487805
Utility - Collision   49.000000  52.333333
Utility - Liability   41.523810        NaN

In [240]:
AgePolicy=pd.pivot_table (df,index='policytype',fill_value=0, aggfunc={'age': np.mean})
AgePolicy

,age
policytype,
Sedan - All Perils,41.644972
Sedan - Collision,38.746060
Sedan - Liability,39.539603
Sport - All Perils,33.545455
Sport - Collision,35.816092
Sport - Liability,21.000000
Utility - All Perils,44.867647
Utility - Collision,49.333333
Utility - Liability,41.523810


In [243]:
AgePolicy.plot(kind = 'area')
plt.title("Edad media por tipo de auto y poliza ", fontsize = 20)
plt.show()


In [253]:
Policyfraude=pd.pivot_table(df,index='policytype',columns='fraudfound_p', aggfunc={'fraudfound_p': np.sum})
Policyfraude

fraudfound_p       
fraudfound_p                    0      1
policytype                              
Sedan - All Perils            0.0  822.0
Sedan - Collision             0.0  768.0
Sedan - Liability             0.0   72.0
Sport - All Perils            0.0    NaN
Sport - Collision             0.0   96.0
Sport - Liability             0.0    NaN
Utility - All Perils          0.0   82.0
Utility - Collision           0.0    6.0
Utility - Liability           0.0    NaN

In [256]:
AgeFraude=pd.pivot_table(df,index='fraudfound_p', aggfunc={'age': np.mean})
AgeFraude

,age
fraudfound_p,
0,39.956957
1,38.265439


### Relaciones

In [263]:
# comprobamos la correlación con la marca y el fraude

df[['make','fraudfound_p']].groupby(['make'], 
                as_index = False).mean().sort_values(by = 'fraudfound_p', ascending = False)

,make,fraudfound_p
10,Mecedes,0.250000
0,Accura,0.125000
16,Saturn,0.103448
15,Saab,0.101852
5,Ford,0.073333
11,Mercury,0.072289
1,BMW,0.066667
6,Honda,0.063906
17,Toyota,0.059596
2,Chevrolet,0.055919


In [264]:
# comprobamos la correlación del area y el fraude

df[['accidentarea','fraudfound_p']].groupby(['accidentarea'], 
                as_index = False).mean().sort_values(by = 'fraudfound_p', ascending = False)

,accidentarea,fraudfound_p
0,Rural,0.083229
1,Urban,0.057155


In [265]:
# comprobamos la correlación del area y el fraude

df[['fault','fraudfound_p']].groupby(['fault'], 
                as_index = False).mean().sort_values(by = 'fraudfound_p', ascending = False)

,fault,fraudfound_p
0,Policy Holder,0.078896
1,Third Party,0.008831


In [266]:
# comprobamos la correlación del tipo de vehículo - seguro y el fraude

df[['policytype','fraudfound_p']].groupby(['policytype'], 
                as_index = False).mean().sort_values(by = 'fraudfound_p', ascending = False)

,policytype,fraudfound_p
4,Sport - Collision,0.137931
6,Utility - All Perils,0.120588
0,Sedan - All Perils,0.100563
7,Utility - Collision,0.100000
1,Sedan - Collision,0.068768
2,Sedan - Liability,0.007219
3,Sport - All Perils,0.000000
5,Sport - Liability,0.000000
8,Utility - Liability,0.000000


In [267]:
# comprobamos la correlación del precio del vehículo y el fraude

df[['vehicleprice','fraudfound_p']].groupby(['vehicleprice'], 
                as_index = False).mean().sort_values(by = 'fraudfound_p', ascending = False)

,vehicleprice,fraudfound_p
4,less than 20000,0.093978
5,more than 69000,0.087338
2,40000 to 59000,0.067245
0,20000 to 29000,0.052110
1,30000 to 39000,0.049533
3,60000 to 69000,0.045977


In [268]:
# comprobamos la correlación del precio del vehículo y el fraude

df[['ageofpolicyholder','fraudfound_p']].groupby(['ageofpolicyholder'], 
                as_index = False).mean().sort_values(by = 'fraudfound_p', ascending = False)

,ageofpolicyholder,fraudfound_p
2,21 to 25,0.148148
1,18 to 20,0.133333
0,16 to 17,0.096875
4,31 to 35,0.064366
8,over 65,0.059055
5,36 to 40,0.058620
3,26 to 30,0.053834
6,41 to 50,0.050919
7,51 to 65,0.050287


### Principales hallazgos:

+ De la base de datos se detectó que cerca del 6% de los incidentes fueron fraudulentos de los cuales la mayor parte tienen un costo de seguro de 400.
+ Respecto a la categoría del vehículo se evidencia un costo más alto del seguro para "Sport" y "Utility", también se obtiene que se presentan más reportes fraudulentos en "Sedan - All Perils".
+ La edad del denunciante tiene inferencia en la categoría del vehículo y el tipo de seguro, adicional la edad media entre las personas que reportan un incidente fraudulento es menor por 1 año en contraste a las que reportan incidentes verdaderos.
+ Frente a las caracteristicas que más se relacionan con los fraudes, se encuentra la marca Mercedes que tiene mayor correlación (25%), el área rural tiene una relación del 83%, por otro lado los vehículos que cuestan menos de 20000 presentó una correlación del 93% y por útimo la edad del propietario con mayor de 21 a 25 tuvó mayor relación frente al fraude con el 15%. 

## Modelo prdictivo

### Preparación de base

In [293]:
df = df.drop(['dayofweekclaimed','monthclaimed', 'weekofmonthclaimed','days_policy_accident','basepolicy'], axis = 1)

KeyError: "['dayofweekclaimed', 'monthclaimed', 'weekofmonthclaimed', 'days_policy_accident', 'basepolicy'] not found in axis"

In [294]:
# Creamos una lista de las variables categoricas
df_categorial = df.select_dtypes(include=["object"])
categories = list(df_categorial.columns)
categories

['monthh',
 'make',
 'accidentarea',
 'fault',
 'policytype',
 'vehiclecategory',
 'vehicleprice',
 'days_policy_claim',
 'pastnumberofclaims',
 'ageofvehicle',
 'ageofpolicyholder',
 'policereportfiled',
 'agenttype',
 'numberofsuppliments',
 'numberofcars']

In [296]:
# Convertimos las variables categoricas en numéricas empleando LabelEncoder
lb = LabelEncoder()

for i in categories:
    df[i] = lb.fit_transform(df[i])

### División del dataset en datos de entrenamiento y datos de test 

Se divide el dataset en datos en 80% datos de entrenamiento y 20% datos de prueba

In [297]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(labels=['fraudfound_p', ], axis=1),df['fraudfound_p'], test_size=0.2, random_state=0)

### Modelo: Árbol de clasificación

In [298]:
model1 = tree.DecisionTreeClassifier()
model1 = model1.fit(X_train,y_train)
# predict with test attributes
y_predict1 = model1.predict(X_test)

fig, ax = plt.subplots(figsize=(12, 5))

print(f"Profundidad del árbol: {model1.get_depth()}")
print(f"Número de nodos terminales: {model1.get_n_leaves()}")

plot = plot_tree(
            decision_tree = model1,
            feature_names = df.drop(columns = 'fraudfound_p').columns,
            class_names   = 'class',
            filled        = True,
            impurity      = False,
            fontsize      = 10,
            precision     = 2,
            ax            = ax
       )

Profundidad del árbol: 29
Número de nodos terminales: 1415


### Evaluación del modelo

In [306]:
print(confusion_matrix(y_test,y_predict1))
print(classification_report(y_test,y_predict1))
print(accuracy_score(y_test, y_predict1))

[[5742   75]
 [  68  283]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5817
           1       0.79      0.81      0.80       351

    accuracy                           0.98      6168
   macro avg       0.89      0.90      0.89      6168
weighted avg       0.98      0.98      0.98      6168

0.9768158236057068


In [312]:
# Random Forest Classifier
from imblearn.ensemble import BalancedRandomForestClassifier
 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
                 

model2 = BalancedRandomForestClassifier(n_estimators = 100, random_state = 0)


In [314]:
# fit model
model2.fit(X_train,y_train)
#predict
y_predict2=model2.predict(X_test)

print("Training Accuracy: ", model2.score(X_train, y_train))
print('Testing Accuarcy: ', model2.score(X_test, y_test))
print(confusion_matrix(y_test,y_predict2))
print(classification_report(y_test,y_predict2))

Training Accuracy:  0.778007457846952
Testing Accuarcy:  0.7420557717250325
[[4236 1581]
 [  10  341]]
              precision    recall  f1-score   support

           0       1.00      0.73      0.84      5817
           1       0.18      0.97      0.30       351

    accuracy                           0.74      6168
   macro avg       0.59      0.85      0.57      6168
weighted avg       0.95      0.74      0.81      6168



### Uso del modelo Random Forest Balanceado

+ Se selecionó el modelo Random Forest que es un algoritmo de aprendizaje supervisado y se balanceo para tener un mejor ajuste este modelo se utilizaría en el negocio dectectanto las posibles caracteristicas del denunciante que conlleven a clasificarlo como un posible fraude y así realizar una investigación más exahustiva del perfil. Adicional serviría para mejora de los motores de reglas, para ver cuáles se pueden mejorar y así poder retroalimentar mejor al modelo.
+ Para medir la efectividad del modelo en el negocio, a parte de considerar el "Accuarcy", se debería implementar un piloto antes del despliegue para evaluar si se disminuyeron las perdidas por fraude frente a los periodos anteriores donde se tenía el modelo, se debe definir la rentabilidad de tener el modelo en produción.
+ ##### Despliegue y producción: 
para obtener las predicciones de los nuevos registros se debe automatizar el proceso y para ello se podría integrar en una arquitectura como AWS donde se podra monitorizar y contar con la trazabilidad que permitá al modelo reentrenarse con los nuevos datos garantizando la estabilidad en la precisión.